In [1]:
import pandas as pd
import numpy as np
import os
import glob
import re as re
import unidecode
pd.options.mode.chained_assignment = None  # default='warn'

Master method that runs all necessary methods in order.

In [2]:
def run_the_script(path):
    masterlist = get_combined_list(path)
    contactlist = contact_list(masterlist)
    masterlist = sum_cols(masterlist)
    masterlist = stats(masterlist)
    masterlist = rename_cols(masterlist)
    masterlist = get_event_results(masterlist, path)
    return masterlist, contactlist

Returns only players who have an email address on file.

In [3]:
def contact_list(df):
    return df[df['PlayerEmail'].notnull()]
    

Goes through a directory, cleans each dataframe and appends it. Returns a dataframe of all cleaned files together.

In [4]:
def get_combined_list(path):
    all_files = glob.glob(path + "/*.csv")
    li = []
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        df = clean_df(df)
        li.append(df)
    masterlist = pd.concat(li, axis=0, ignore_index=True)
    return masterlist

Cleans up dataframe

In [5]:
def clean_df(bigdf):
    df = bigdf.copy()
    df = drop_columns(df)
    df = remove_all_extra_spaces(df)
    df = remove_chars(df)
    df = all_players(df)
    df = add_attendance(df)
    return df

All Scoreholio bracket standing outputs contain the same columns. This drops GameID, GameName, TeamID, TeamName, Player1User, Player2User, all Player3 and Player4 columns, and Seed.

In [6]:
def drop_columns(df):
    return df.iloc[:, np.r_[5:9,10:14,22:25]]

Remove all extra spaces. Helps to prevent duplicate player names from remaining in the dataframe due to an errant space inputted in the front of back of a name.

In [7]:
def remove_all_extra_spaces(df):
    return df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

Some organizers put funny names for players where they replace letters with characters. We remove the characters for a more consistent naming convention.

In [8]:
def remove_chars(df):
    #unidecode handles characters that are not part of the 26 letter alphabet such as accents.
    return df.applymap(lambda x: unidecode.unidecode(x) if isinstance(x, str) else x)

Drops columns with no information in them.

In [9]:
def drop_nan_columns(df):
    return df[df.columns[~df.isnull().all()]]

Gets list of all player names.

In [10]:
def all_players(df):
    player1 = df.iloc[:, np.r_[0:4,8:11]]
    player2 = df.iloc[:, np.r_[4:11]]

    player1, player2 = make_players(player1,player2)
    
    finaldf = pd.concat([player1,player2], ignore_index=True)
    
    cols = list(finaldf.columns)
    cols = [cols[-1]] + cols[:-1]
    finaldf = finaldf[cols]
    
    return finaldf

In [11]:
def make_players(player1,player2):
    
    player1 = make_name(player1)
    player2 = make_name(player2)

    player1 = fill_name(player1)
    player2 = fill_name(player2)
    
    player1=remove_non_alph_num(player1)
    player2=remove_non_alph_num(player2)

    player1=drop_cols(player1)
    player2=drop_cols(player2)
    
    player1=rename_player_cols(player1)
    player2=rename_player_cols(player2)
    
    return player1, player2

Combines first and last name of player.

In [12]:
def make_name(df):
    df["Player Name"] = df.iloc[:,1]+" " + df.iloc[:,2]
    return df

In [13]:
def fill_name(df):
    df["Player Name"] = df.iloc[:,0].fillna(df.iloc[:,0])
    return df

In [14]:
def remove_non_alph_num(df):
    df['Player Name']=df.iloc[:,0].str.replace('[^\w\s]','', regex=True)
    return df

In [15]:
def drop_cols(df):
    return df.drop(df.columns[0:3], axis=1)

Adds attendance value for a player for the event attended.

In [16]:
def add_attendance(df):
    df['Attendance'] = 1
    return df

Renames column to PlayerEmail

In [17]:
def rename_player_cols(df):
    df = df.rename(columns={df.columns[0]: 'PlayerEmail'})
    return df

This groups by "Player Name" column and sums wins, losses, attendance, max wins in one even (Wins2) and gets the player's best placement out of all events (Place).

In [18]:
def sum_cols(df):
    df = df.groupby(["Player Name"], dropna=False).agg(
     Wins = ("Wins",'sum'),
     Losses = ("Losses",'sum'),
     Attendance = ("Attendance",'sum'),
     Wins2 = ("Wins",'max'),
     Place = ("Place",'min'),
     ).reset_index().sort_values(by=["Player Name"])
    return df

Calculcates player's win/loss rate.

In [19]:
def stats(df):
    df["Win Rate"] = (df["Wins"]/(df["Wins"]+df["Losses"]))
    return df

Rename more columns

In [20]:
def rename_cols(df):
    df=df.rename(columns={"Wins2": "Most Wins in One Event", "Place": "Best Placement"})
    return df

This goes through the path given which contains all Scoreholio event outputs and merges them into one large dataframe.

In [21]:
def get_event_results(masterlist, path):
    all_files = glob.glob(path + "/*.csv")
    li = []
    event = 1
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        df = clean_df(df)
        df = df.iloc[:, 0:4]
        df = rename_event_cols(df, event, str(filename))
        if(event == 1):
            finallist = pd.merge(masterlist,df,on="Player Name", how='left')
        else:
            finallist = pd.merge(finallist,df,on="Player Name", how='left')
        event= event+1
    return finallist

Renames event columns to better show which event each column is referring to.

In [22]:
def rename_event_cols(df, event, filename):
    eventname = renameevent(filename, event)
    df=df.rename(columns={"Place": eventname + " Placement", "Wins": eventname + " Wins", "Losses": eventname + " Losses"})
    return df

Gets file name for each event.

In [23]:
def renameevent(filename, event):
    filename = str(filename)
    word_list = filename.split("\\")  # list of words
    filename = word_list[-1]
    filename = filename.replace(".csv",'')
    return filename